In [37]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, mmread
import sys
import itertools 
import pickle
import matplotlib.pylab as plt
import time

In [38]:
%matplotlib inline

In [39]:
import rankingmethod
import rankingmeasure
import rank

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from  experiment import *

# loading data sets

In [42]:
data = pd.read_csv("../datasets/soc-Epinions1.txt", sep='\t', header=4)
data = data.as_matrix()

In [43]:
epinion, epinion_shape = make_valid_data(data)

In [44]:
data = pd.read_csv("../datasets/Slashdot0811.txt", sep='\t', header=4)
data = data.as_matrix()

In [45]:
slashdot, slashdot_shape = make_valid_data(data)

In [46]:
data =  pd.read_csv("../datasets/ml-100k/u.data", sep='\t', header=None).ix[:, :1]
data = data.as_matrix()

In [47]:
ml100k, ml100k_shape = make_valid_data(data)

In [48]:
data = pd.read_csv("../datasets/ml-1m/ratings.dat", sep='::', header=None,  engine='python').ix[:, :1]
data = data.as_matrix()

In [49]:
ml1m, ml1m_shape = make_valid_data(data)

### Зададим параметры, которые были выбраны при помощивалидации

In [50]:
def create_list_cf(user_item):
    list_cf = [
        rankingmethod.PopRec(user_item),
#         rankingmethod.CLiMF(user_item, K=10, reg=0.001, lrate=0.01, verbose=0, maxiter=10),
        
#         rank.CLiMF(user_item, K=5, reg=0.01, lrate=0.1, verbose=0, maxiter=10),
#         rank.CLiMF(user_item, K=5, reg=0.01, lrate=0.01, verbose=0, maxiter=10),
#         rank.CLiMF(user_item, K=5, reg=0.01, lrate=0.001, verbose=0, maxiter=10),
        
        rank.CLiMF(user_item, K=10, reg=0.005, lrate=0.001, verbose=0, maxiter=15),
#         rank.BPR_MF(user_item, K = 10, lrate=0.012, regIpos=0.012, 
#                                   regIneg=0.012, regU=0.012, maxiter=60, verbose=0 ),
#         rank.BPR_MF(user_item, K = 10, lrate=0.02, regIpos=0.012, 
#                                   regIneg=0.012, regU=0.012, maxiter=60, verbose=0 ),
        rank.BPR_MF(user_item, K = 10, lrate=0.05, regIpos=0.0025, 
                                  regIneg=0.0025, regU=0.0025, maxiter=300, verbose=0 ),
        
#         rank.CLiMF(user_item, K=10, reg=0.001, lrate=0.1, verbose=0, maxiter=10),
#         rank.CLiMF(user_item, K=10, reg=0.001, lrate=0.001, verbose=0, maxiter=10),
#         rankingmethod.BPR_MF(user_item, K = 20, lrate=0.012, regIpos=0.012, 
#                                   regIneg=0.012, regU=0.012, maxiter=4, verbose=0 ),
#          rankingmethod.iMF(user_item, K=10, lmbd=0.01, alpha=40, maxiter=30, verbose=0),
        
         rankingmethod.iMF(user_item, K=10, lmbd=0.015, alpha=1, maxiter=15, verbose=0),
#         rankingmethod.TFMAP(user_item, lrate=0.01, maxiter=10),
        rank.TFMAP(user_item, lrate=0.01, maxiter=20)
    ]
    return list_cf#[list_cf[i] for i in index]

### Проведение такого же эксперимента, как у создатей CLiMF(Точнее похожего, т.к. было решено не удалять топ 3 всех предметов. И даже с этим удалением получить такие же цифры, что авторы epinion не получилось)

In [51]:
def climf_experiment(data, list_cf, K=5, maxiter=1, topK=5, verbose=0, validation_type=1):
    res = np.zeros((6, len(list_cf)))
    for t in range(maxiter):
        if validation_type == 0:
            test, train, traintest = givenK_train_test(data, K)
        elif validation_type == 1:
            test, train, traintest = ratio_train_test(data, K)
        for i, cf in enumerate(list_cf):
            start_time = time.time()
            cf.fit(train)
#             print(time.time() - start_time)
            prec_k, one_recal_k, ngdc, mp = rankingmeasure.get_Kmetrics(train, test, cf, K=topK)
            mrr, auc = rankingmeasure.get_listmetrics(train, test, cf)
            res[:, i] += np.array([prec_k, one_recal_k, ngdc, mp, mrr, auc])
            if verbose == 1:
                print(t, i, res[:, i] / (t + 1))
                sys.stdout.flush()
    res /= maxiter
    res = pd.DataFrame(res)
    
    res.columns = [type(cf).__name__ for cf in list_cf]
    res.index = [name + str(topK) for name in ['prec@', '1recal@', 'NDCG@', 'MAP@']] + ['MRR', 'AUC']
#     res.columns = ['PopRec', 'CLiMF', 'BRP_MP', 'iMF', 'TFMAP']
#     res.index = ['prec@' + str(topK),'1recal@' + str(topK), 'NDCG@' + str(topK), 
#                'MAP@' + str(topK), 'MRR', 'AUC']
    return res

In [52]:
climf_experiment(ml100k, create_list_cf(ml100k_shape), K =0.1, maxiter=1, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.139950,0.142928,0.267246,0.297270,0.147891
1recal@5,0.451613,0.456576,0.694789,0.733251,0.435484
NDCG@5,0.154961,0.159296,0.283737,0.322085,0.165271
MAP@5,0.285389,0.293457,0.451184,0.504382,0.288277
MRR,0.341340,0.345123,0.502585,0.556852,0.338402
AUC,0.859741,0.847079,0.931770,0.931023,0.850399


In [53]:
climf_experiment(ml100k, create_list_cf(ml100k_shape), K =0.2, maxiter=1, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.240199,0.246898,0.394541,0.447643,0.257568
1recal@5,0.662531,0.655087,0.846154,0.888337,0.615385
NDCG@5,0.259162,0.261185,0.411810,0.478220,0.274834
MAP@5,0.424500,0.417957,0.581512,0.665104,0.416967
MRR,0.480649,0.471118,0.633542,0.720579,0.462428
AUC,0.856661,0.841716,0.929072,0.928343,0.847104


In [54]:
climf_experiment(epinion, create_list_cf(epinion_shape), K =0.1, maxiter=1, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.031782,0.030375,0.055755,0.076686,0.029694
1recal@5,0.141203,0.133712,0.226788,0.288990,0.130760
NDCG@5,0.036236,0.035332,0.058648,0.084550,0.034052
MAP@5,0.083203,0.081563,0.121041,0.171857,0.078353
MRR,0.106883,0.106103,0.154131,0.203232,0.102533
AUC,0.869996,0.833470,0.919266,0.896619,0.867311


In [55]:
climf_experiment(epinion, create_list_cf(epinion_shape), K =0.2, maxiter=1, topK=5,  verbose=1)

0 0 [ 0.0600227   0.2492622   0.06807169  0.14936594  0.18106551  0.87154629]
0 1 [ 0.058479    0.24245176  0.06726779  0.14920545  0.18087421  0.82344455]
0 2 [ 0.0999773   0.35323496  0.10529264  0.19731429  0.23854742  0.91856299]
0 3 [ 0.13593644  0.43950057  0.14624658  0.26657901  0.3050251   0.89413977]
0 4 [ 0.05471056  0.23087401  0.06345845  0.1434027   0.17469081  0.866994  ]


,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.060023,0.058479,0.099977,0.135936,0.054711
1recal@5,0.249262,0.242452,0.353235,0.439501,0.230874
NDCG@5,0.068072,0.067268,0.105293,0.146247,0.063458
MAP@5,0.149366,0.149205,0.197314,0.266579,0.143403
MRR,0.181066,0.180874,0.238547,0.305025,0.174691
AUC,0.871546,0.823445,0.918563,0.894140,0.866994


In [56]:
climf_experiment(slashdot, create_list_cf(slashdot_shape), K=0.1, maxiter=1, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.014960,0.015475,0.024743,0.043965,0.014531
1recal@5,0.069937,0.072511,0.104548,0.166476,0.066505
NDCG@5,0.017404,0.017257,0.026554,0.048168,0.015411
MAP@5,0.041657,0.040385,0.056634,0.097097,0.034294
MRR,0.059544,0.057325,0.077540,0.121921,0.050586
AUC,0.849921,0.795534,0.871066,0.853451,0.843581


In [57]:
climf_experiment(slashdot, create_list_cf(slashdot_shape), K=0.2, maxiter=1, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.029863,0.029348,0.042048,0.074485,0.028175
1recal@5,0.131150,0.129148,0.160898,0.255578,0.124142
NDCG@5,0.033434,0.032729,0.043991,0.080004,0.030640
MAP@5,0.074740,0.073377,0.086740,0.149358,0.067775
MRR,0.100142,0.097755,0.115010,0.181893,0.091801
AUC,0.843574,0.778659,0.863101,0.846316,0.836296


In [58]:
climf_experiment(ml1m, create_list_cf(ml1m_shape), K=0.1, maxiter=1, topK=5, verbose=1)

0 0 [ 0.1336457   0.41124527  0.14399806  0.2564864   0.29679144  0.85737817]
0 1 [ 0.11573256  0.36925572  0.11985524  0.20916582  0.25224184  0.84890599]
0 2 [ 0.22375203  0.61416471  0.24006127  0.39429927  0.44244401  0.92443239]
0 3 [ 0.2600829   0.65200937  0.27999383  0.44248588  0.48957763  0.91715365]
0 4 [ 0.07925752  0.28707875  0.08380195  0.16032644  0.19862019  0.84915509]


,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.133646,0.115733,0.223752,0.260083,0.079258
1recal@5,0.411245,0.369256,0.614165,0.652009,0.287079
NDCG@5,0.143998,0.119855,0.240061,0.279994,0.083802
MAP@5,0.256486,0.209166,0.394299,0.442486,0.160326
MRR,0.296791,0.252242,0.442444,0.489578,0.198620
AUC,0.857378,0.848906,0.924432,0.917154,0.849155


In [59]:
climf_experiment(ml1m, create_list_cf(ml1m_shape), K=0.2, maxiter=1, topK=5, verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.226563,0.212507,0.334547,0.397225,0.140530
1recal@5,0.577942,0.559560,0.753109,0.801045,0.442062
NDCG@5,0.238727,0.218131,0.350127,0.417528,0.150439
MAP@5,0.378080,0.339910,0.508567,0.576701,0.268975
MRR,0.418982,0.385665,0.558877,0.625551,0.314291
AUC,0.857999,0.852355,0.922871,0.915748,0.850259


### Построение ансамбля

# Netflix ensemble
# Regression

In [60]:
from sklearn import cross_validation
import rankensemble
import rankingmeasure

In [61]:
def netflix_ensemble_experiment(data, list_cf, list_ensemble, Ktrain=0.1, Kvalidation=0.1, 
                                     maxiter=1, topK=5,  verbose=0):
    res = np.zeros((4, len(list_cf) + len(list_ensemble)))
    for t in range(maxiter):
        test, train, traintest = ratio_train_test(data, Ktrain + Kvalidation)
        validation, train, trainvalidation = ratio_train_test(train, Kvalidation, data_is_xlist=True)
        
        for i in range(len(list_cf)):
            list_cf[i].fit(train)
            prec_k, one_recal_k, ngdc, mp = rankingmeasure.get_Kmetrics(trainvalidation, test, list_cf[i], K=topK)
            res[:, i] += np.array([prec_k, one_recal_k, ngdc, mp])
            if verbose == 1:
                print(t, i, res[:, i] / (t + 1))
                sys.stdout.flush()
        for j, ens in enumerate(list_ensemble):
            m = j + len(list_cf)
            ens.fit(list_cf, validation, train, trainvalidation)
#             return X, y
            prec_k, one_recal_k, ngdc, mp = rankingmeasure.get_Kmetrics(trainvalidation, test, ens, K=topK)
            res[:, m] += np.array([prec_k, one_recal_k, ngdc, mp])
            if verbose == 1:
                print(t,  m, res[:, m] / (t + 1))
#         for j, ens in enumerate(list_ensemble):
#             m = j + len(list_cf)
#             ens.fit(list_cf)
#             prec_k, one_recal_k, ngdc, mp = rankingmeasure.get_Kmetrics(train, test, ens, K=topK)
#             res[:, m] += np.array([prec_k, one_recal_k, ngdc, mp])
#             if verbose == 1:
#                 print(t, m, res[:, m] / (t +1))
        
    res /= maxiter
    res[:, len(list_cf) - 1] = np.max(res[:, :len(list_cf)], axis=1)
    res = res[:, len(list_cf)- 1:]
    res = pd.DataFrame(res)
    res.columns = ["best_result"] + [type(ens).__name__ for ens  in list_ensemble]
    res.index = [name + str(topK) for name in ['prec@', '1recal@', 'NDCG@', 'MAP@']]
#     res.index = []
#     res.columns = ['PopRec', 'CLiMF', 'BRP_MP', 'iMF', 'TFMAP', 'top_ensemble', 'rating_ensamble']
#     res.index = ['prec@' + str(topK),'1recal@' + str(topK), 'NDCG@' + str(topK), 
#                'MAP@' + str(topK)]
    return res

In [62]:
from sklearn import linear_model
import rankensemble
from sklearn import ensemble
from sklearn.kernel_ridge import KernelRidge

In [63]:
import copy

In [64]:
list_ensemble = [
    rankensemble.regression_ensemble(linear_model.Ridge(alpha=100)),
    rankensemble.inner_value_ensemble(),
    rankensemble.boost_val_ens(rankingmeasure.get_NDCG,[3,0,1,2], num_weights=11),
    rankensemble.tree_val_ens(rankingmeasure.get_NDCG, [[3, 0],[1,2]], num_weights=11)
]

In [65]:
netflix_ensemble_experiment(ml100k, create_list_cf(ml100k_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=5)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.446452,0.278114,0.392258,0.451315,0.453201
1recal@5,0.879901,0.667494,0.822333,0.880149,0.881390
NDCG@5,0.472480,0.298063,0.420762,0.477443,0.479024
MAP@5,0.652318,0.454898,0.603423,0.655568,0.655539


In [66]:
netflix_ensemble_experiment(ml100k, create_list_cf(ml100k_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=1, maxiter=3)

0 0 [ 0.32084367  0.7146402   0.34645261  0.50476978]
0 1 [ 0.4662531   0.87344913  0.48138633  0.62882203]
0 2 [ 0.54987593  0.93052109  0.5697363   0.71664082]
0 3 [ 0.34615385  0.69851117  0.36220939  0.49494762]
0 4 [ 0.39305211  0.78908189  0.41040393  0.55415116]
0 5 [ 0.49702233  0.89205955  0.52224099  0.67765715]
0 6 [ 0.56575682  0.93052109  0.58633574  0.72654742]
0 7 [ 0.56253102  0.92679901  0.58647862  0.73115006]
1 0 [ 0.31501241  0.69789082  0.33950687  0.49485026]
1 1 [ 0.45856079  0.86972705  0.47520206  0.62937776]
1 2 [ 0.54416873  0.92307692  0.56598803  0.71362783]
1 3 [ 0.33548387  0.69168734  0.35450349  0.49172526]
1 4 [ 0.37245658  0.7630273   0.39221528  0.53962297]
1 5 [ 0.49069479  0.88151365  0.51345251  0.66469793]
1 6 [ 0.55521092  0.92431762  0.57722547  0.72074459]
1 7 [ 0.55334988  0.92183623  0.57834767  0.72655345]
2 0 [ 0.31381307  0.70057899  0.33864788  0.49689482]
2 1 [ 0.45690653  0.86807279  0.47314025  0.62621887]
2 2 [ 0.54160463  0.9280397 

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.541605,0.372787,0.489992,0.555418,0.547891
1recal@5,0.928040,0.756824,0.881720,0.928453,0.923077
NDCG@5,0.563852,0.391981,0.513989,0.577347,0.572760
MAP@5,0.714177,0.537806,0.667333,0.722169,0.723996


In [67]:
netflix_ensemble_experiment(epinion, create_list_cf(epinion_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.129035,0.082361,0.098252,0.132577,0.135982
1recal@5,0.420885,0.304200,0.350057,0.429058,0.432236
NDCG@5,0.138074,0.089173,0.103557,0.140958,0.143964
MAP@5,0.251218,0.175528,0.195928,0.254885,0.256035


In [68]:
netflix_ensemble_experiment(epinion, create_list_cf(epinion_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.170715,0.104745,0.137980,0.177072,0.182883
1recal@5,0.510783,0.374801,0.455846,0.527582,0.533939
NDCG@5,0.181573,0.113291,0.144062,0.188282,0.193359
MAP@5,0.312035,0.219128,0.259211,0.323588,0.326621


In [69]:
netflix_ensemble_experiment(ml1m, create_list_cf(ml1m_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.396720,0.178555,0.359849,0.400793,0.400721
1recal@5,0.807172,0.494143,0.770049,0.812579,0.813300
NDCG@5,0.415989,0.182305,0.370419,0.420478,0.420210
MAP@5,0.575993,0.292261,0.516833,0.581903,0.581216


In [70]:
netflix_ensemble_experiment(ml1m, create_list_cf(ml1m_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.487007,0.216111,0.443251,0.490899,0.481781
1recal@5,0.866823,0.564246,0.838169,0.872049,0.857272
NDCG@5,0.505163,0.221250,0.455997,0.509798,0.502215
MAP@5,0.652758,0.343543,0.600495,0.658076,0.651395


In [71]:
netflix_ensemble_experiment(slashdot, create_list_cf(slashdot_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.075057,0.034296,0.050572,0.075029,0.075915
1recal@5,0.254005,0.147883,0.199943,0.256293,0.258867
NDCG@5,0.080496,0.036987,0.053533,0.080918,0.081579
MAP@5,0.148947,0.080431,0.108799,0.151041,0.152045


In [72]:
netflix_ensemble_experiment(slashdot, create_list_cf(slashdot_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.101487,0.053804,0.073713,0.101831,0.103947
1recal@5,0.324657,0.217677,0.277174,0.325372,0.330664
NDCG@5,0.108359,0.058386,0.079002,0.108754,0.110665
MAP@5,0.195109,0.122139,0.156776,0.195859,0.198367


In [73]:
list_ensemble = [
    rankensemble.tree_val_ens(rankingmeasure.get_prec_K, [[3, 0],[1,2]], num_weights=11),
    rankensemble.tree_val_ens(rankingmeasure.get_one_recal_K, [[3, 0],[1,2]], num_weights=11),
    rankensemble.tree_val_ens(rankingmeasure.get_MAP, [[3, 0],[1,2]], num_weights=11),
    rankensemble.tree_val_ens(rankingmeasure.get_NDCG, [[3, 0],[1,2]], num_weights=11)
]

In [74]:
netflix_ensemble_experiment(ml100k, create_list_cf(ml100k_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=1, maxiter=1)

0 0 [ 0.24218362  0.65260546  0.26465323  0.43577164]
0 1 [ 0.37816377  0.82258065  0.39086552  0.55239178]
0 2 [ 0.44739454  0.88833747  0.47145034  0.64708092]
0 3 [ 0.25310174  0.62282878  0.26833271  0.40971016]
0 4 [ 0.44987593  0.89081886  0.47426476  0.65185243]
0 5 [ 0.45806452  0.89205955  0.48606692  0.66681141]
0 6 [ 0.45459057  0.88709677  0.48284741  0.66364592]
0 7 [ 0.45781638  0.88709677  0.48482568  0.66538462]


,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.447395,0.449876,0.458065,0.454591,0.457816
1recal@5,0.888337,0.890819,0.892060,0.887097,0.887097
NDCG@5,0.471450,0.474265,0.486067,0.482847,0.484826
MAP@5,0.647081,0.651852,0.666811,0.663646,0.665385


In [75]:
netflix_ensemble_experiment(ml100k, create_list_cf(ml100k_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.524069,0.536725,0.536725,0.536725,0.536725
1recal@5,0.915633,0.929280,0.929280,0.929280,0.929280
NDCG@5,0.548121,0.561869,0.561869,0.561869,0.561869
MAP@5,0.707239,0.718166,0.718166,0.718166,0.718166


In [76]:
netflix_ensemble_experiment(epinion, create_list_cf(epinion_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.128263,0.133893,0.132395,0.133121,0.133893
1recal@5,0.419750,0.441317,0.433598,0.441090,0.441317
NDCG@5,0.137557,0.143103,0.141807,0.140842,0.143103
MAP@5,0.251110,0.261575,0.259145,0.256982,0.261575


In [77]:
netflix_ensemble_experiment(epinion, create_list_cf(epinion_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.168173,0.175437,0.178479,0.175074,0.175437
1recal@5,0.500795,0.520772,0.527809,0.533031,0.520772
NDCG@5,0.178086,0.185403,0.187789,0.184682,0.185403
MAP@5,0.304935,0.317274,0.320160,0.320722,0.317274


In [78]:
netflix_ensemble_experiment(ml1m, create_list_cf(ml1m_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.395423,0.400144,0.400396,0.400000,0.400144
1recal@5,0.800144,0.806271,0.807353,0.807172,0.806271
NDCG@5,0.415935,0.420379,0.420517,0.420004,0.420379
MAP@5,0.576955,0.581040,0.581251,0.581108,0.581040


In [79]:
netflix_ensemble_experiment(ml1m, create_list_cf(ml1m_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.486754,0.491476,0.491908,0.488737,0.491692
1recal@5,0.862858,0.874932,0.871328,0.872229,0.872229
NDCG@5,0.505064,0.509525,0.510863,0.507341,0.509243
MAP@5,0.651736,0.656594,0.658771,0.654780,0.655432


In [80]:
netflix_ensemble_experiment(slashdot, create_list_cf(slashdot_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.075830,0.076659,0.076659,0.076745,0.076745
1recal@5,0.253576,0.258295,0.258295,0.259868,0.259868
NDCG@5,0.081787,0.082365,0.082365,0.082489,0.082489
MAP@5,0.150924,0.151986,0.151986,0.152748,0.152748


In [81]:
netflix_ensemble_experiment(slashdot, create_list_cf(slashdot_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=1)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.099914,0.102860,0.102860,0.102860,0.102860
1recal@5,0.322368,0.331808,0.331808,0.331808,0.331808
NDCG@5,0.106621,0.109790,0.109790,0.109790,0.109790
MAP@5,0.191864,0.198105,0.198105,0.198105,0.198105
